In [ ]:
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_link = "/content/drive/MyDrive/UIT_Fact_Checking_Contest/Data/Processed_public_data/public_processed_train.json"

In [ ]:
!pip install rank_bm25

In [ ]:
from rank_bm25 import BM25Okapi

def get_embedding(corpus):
  tokenized_corpus = [doc.split(" ") for doc in corpus]

  bm25 = BM25Okapi(tokenized_corpus)
  return bm25

In [ ]:
import random
from tqdm import tqdm

list_sent1 = []
list_sent2 = []
list_label = []

for key in tqdm(data.keys()):
  dt = data[key]
  while True:
    key_not_info = random.choice(list(data.keys()))
    if key_not_info != key: break
  context = dt['context'].split('\n\n')
  evidence = dt['evidence']
  claim = dt['claim']
  tokenized_query = evidence.split(" ")

  all_embedding = get_embedding(context)
  # texts = corpus_embedding.get_top_n(tokenized_query, corpus, n=30)
  score = all_embedding.get_scores(tokenized_query)

  top_3_max_indices = sorted(range(len(score)), key=lambda i: score[i], reverse=True)[:3]

  flag = False
  for idx in top_3_max_indices:
    list_sent1.append(claim)
    list_sent2.append(context[idx])

    if flag == False:
      list_label.append(1)
      flag = True
    else:
      list_label.append(0)

  #nei_info
  evidence_nei = data[key_not_info]['evidence']
  list_sent1.append(claim)
  list_sent2.append(evidence_nei)
  list_label.append(0)  

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({
    "sent1" : list_sent1,
    "sent2" : list_sent2,
    "label" : list_label
})

In [ ]:
df.to_csv("/content/drive/MyDrive/UIT_Fact_Checking_Contest/Data/Processed_public_data/rerank_dataset.csv",index=False,escapechar='\\')